### Evaluate model

In [ ]:
import matplotlib.pyplot as plt
import torch

from datasets import load_dataset
from torchvision.transforms import ToPILImage
from transformers import DetrForObjectDetection, DetrImageProcessor

from Cocordiais import CocordiaisDataset, CocordiaisUtils
from PIL import Image as PImage

In [ ]:
MODEL_NAMES = [
  "detr-cordiais-no-aug-48",
  "detr-cordiais-aug-48",
  "detr-cordiais-aug-64",
  "detr-cordiais-no-aug-100",
  "detr-cordiais-aug-100"
]

HF_MODEL= f"thiagohersan/{MODEL_NAMES[0]}"

HF_DATASET = "thiagohersan/cordiais-faces"

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
detr_model = DetrForObjectDetection.from_pretrained(HF_MODEL, id2label=CocordiaisUtils.ID2LABEL)
detr_processor = DetrImageProcessor.from_pretrained(HF_MODEL)

In [ ]:
hf_dataset = load_dataset(HF_DATASET)
hf_dataset_train = hf_dataset["train"].train_test_split(test_size=0.2, shuffle=True, seed=101010)

dataset_test = CocordiaisDataset(hf_dataset["test"], img_processor=detr_processor, train=False)
dataset_eval = CocordiaisDataset(hf_dataset_train["test"], img_processor=detr_processor, train=False)

print("Number of examples:\n  Test: %s\n  Evaluation: %s" % (len(dataset_test), len(dataset_eval)))

In [ ]:
PLOT_COLORS = [
  [0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
  [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]
]

def plot_results(pil_img, results):
  scores = results["scores"]
  labels = results["labels"]
  boxes = results["boxes"]

  plt.figure(figsize=(16,10))
  plt.imshow(pil_img)
  ax = plt.gca()
  colors = PLOT_COLORS * 100
  for score, label, (xmin, ymin, xmax, ymax),c  in zip(scores.tolist(), labels.tolist(), boxes.tolist(), colors):
    ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, fill=False, color=c, linewidth=3))
    text = f'{CocordiaisUtils.ID2LABEL[label]}: {score:0.2f}'
    ax.text(xmin, ymin, text, fontsize=15, bbox=dict(facecolor='yellow', alpha=0.5))
  plt.axis('off')
  plt.show()

In [ ]:
data_idx = 20
pixel_values, pixel_mask, labels = dataset_eval.data[data_idx].values()
pixel_values = pixel_values.unsqueeze(0).to(device)
print(pixel_values.shape)

In [ ]:
detr_model.to(device)
with torch.no_grad():
  outputs = detr_model(pixel_values=pixel_values, pixel_mask=None)

In [ ]:
image_id = labels["image_id"].item()
image = ToPILImage()(pixel_values.squeeze())

width, height = image.size
results = detr_processor.post_process_object_detection(
  outputs,
  target_sizes=[(height, width)],
  threshold=0.5
)
plot_results(image, results[0])